In [1]:
# Import the required libraries.
import os
import cv2
import math
import random
import numpy as np
import datetime as dt
import tensorflow as tf
from collections import deque
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

from tensorflow.keras.layers import *
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import plot_model

from tensorflow.keras.applications import Xception
from tensorflow.keras.models import Model
from tensorflow.keras.layers import LSTM, Dense, TimeDistributed, Flatten
from tensorflow.keras.layers import Dropout

import warnings
warnings.filterwarnings('ignore')

2025-06-29 10:20:17.603414: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1751192417.857851      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1751192417.942105      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
# set seeed to get similar values
seed_constant = 27
np.random.seed(seed_constant)
random.seed(seed_constant)
tf.random.set_seed(seed_constant)

In [3]:
# Specify the directory containing the UCF50 dataset
DATASET_DIR = "/kaggle/input/ucf50/UCF50"

os.path.exists(DATASET_DIR)

True

In [4]:
all_class_names = os.listdir(DATASET_DIR)

len(all_class_names)

50

In [5]:
# Global constant variables -> 
NO_OF_CLASSES = 6
CLASSES_LIST = all_class_names[:NO_OF_CLASSES]

# Model Configuration
IMAGE_HEIGHT, IMAGE_WIDTH = 224, 224
SEQUENCE_LENGTH = 15

# set drop out rate
DROPOUT_RATE = 0.2

# set datas
MAX_VIDEO_PER_CLASS = 100

# split dataset
TEST_SIZE = 0.20

# model fit parameters
EPOCHS = 50
BATCH_SIZE = 6
VALIDATION_SPLIT = 0.20


# give a name of the model to save
MODEL_NAME = "Xception"

In [6]:
# # Create a Matplotlib figure and specify the size of the figure
# plt.figure(figsize=(20, 20))

# # Get the names of all classes/categories in UCF50
# all_classes_names = os.listdir(DATASET_DIR)

# # Check if the directory contains classes
# if not all_classes_names:
#     raise ValueError(f"No classes found in {DATASET_DIR}. Please ensure the dataset is extracted correctly.")

# # Generate a list of 20 random values, ensuring we don't sample more than available classes
# random_range = random.sample(range(len(all_classes_names)), min(20, len(all_classes_names)))

# # Iterating through all the generated random values
# for counter, random_index in enumerate(random_range, 1):
#     selected_class_Name = all_classes_names[random_index]
#     video_files_names_list = os.listdir(f'{DATASET_DIR}/{selected_class_Name}')
#     selected_video_file_name = random.choice(video_files_names_list)
#     video_reader = cv2.VideoCapture(f'{DATASET_DIR}/{selected_class_Name}/{selected_video_file_name}')
#     _, bgr_frame = video_reader.read()
#     video_reader.release()
#     rgb_frame = cv2.cvtColor(bgr_frame, cv2.COLOR_BGR2RGB)
#     cv2.putText(rgb_frame, selected_class_Name, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
#     plt.subplot(5, 4, counter)
#     plt.imshow(rgb_frame)
#     plt.axis('off')
# plt.show()

In [7]:
# frame extraction
import cv2
import numpy as np
import os

def frames_extraction(video_path, 
                      sequence_length=SEQUENCE_LENGTH, 
                      image_height=IMAGE_HEIGHT, 
                      image_width=IMAGE_WIDTH):
    # Declare a list to store video frames
    frames_list = []

    # Check if video file exists
    if not os.path.exists(video_path):
        print(f"Error: Video file not found at {video_path}")
        return None

    # Read the video file using VideoCapture
    video_reader = cv2.VideoCapture(video_path)

    # Check if the video was opened successfully
    if not video_reader.isOpened():
        print(f"Error: Could not open video file {video_path}")
        video_reader.release()
        return None

    # Get the total number of frames in the video
    video_frames_count = int(video_reader.get(cv2.CAP_PROP_FRAME_COUNT))

    # Ensure the video has enough frames
    if video_frames_count < sequence_length:
        print(f"Warning: Video {video_path} has only {video_frames_count} frames, less than required {sequence_length}")
        video_reader.release()
        return None

    # Calculate the interval after which frames will be sampled
    skip_frames_window = max(int(video_frames_count / sequence_length), 1)

    # Iterate to extract the specified number of frames
    for frame_counter in range(sequence_length):
        # Set the current frame position
        video_reader.set(cv2.CAP_PROP_POS_FRAMES, frame_counter * skip_frames_window)

        # Read the frame
        success, frame = video_reader.read()

        # Check if the frame was read successfully
        if not success or frame is None:
            print(f"Warning: Failed to read frame {frame_counter} from {video_path}")
            break

        # Resize the frame to the specified dimensions
        try:
            resized_frame = cv2.resize(frame, (image_width, image_height))
        except Exception as e:
            print(f"Error resizing frame {frame_counter} from {video_path}: {e}")
            break

        # Normalize the frame to [0, 1] for model input
        normalized_frame = resized_frame / 255.0

        # Append the normalized frame to the list
        frames_list.append(normalized_frame)

    # Release the VideoCapture object
    video_reader.release()

    # Ensure the correct number of frames is extracted
    if len(frames_list) != sequence_length:
        print(f"Warning: Extracted {len(frames_list)} frames instead of {sequence_length} from {video_path}")
        return None

    # Convert to numpy array for consistency
    frames_list = np.array(frames_list)

    return frames_list

In [8]:
# RUN create dataset function definition

import os
import numpy as np

def create_dataset(dataset_dir,
                   classes_list, 
                   sequence_length=SEQUENCE_LENGTH, 
                   image_height=IMAGE_HEIGHT, 
                   image_width=IMAGE_WIDTH, 
                   max_videos_per_class=None):
    '''
    This function extracts data for selected classes and creates the dataset.
    Args:
        dataset_dir: The directory containing the UCF-50 dataset (e.g., "/kaggle/working/UCF50_dataset/UCF50").
        classes_list: List of class names to include in the dataset.
        sequence_length: Number of frames to extract per video (default: 20).
        image_height: Height to resize frames to (default: 224 for Xception).
        image_width: Width to resize frames to (default: 224 for Xception).
        max_videos_per_class: Maximum number of videos to process per class (optional, for testing).
    Returns:
        features: A numpy array of extracted frame sequences with shape (n_videos, sequence_length, image_height, image_width, 3).
        labels: A numpy array of class indexes.
        video_files_paths: A list of video file paths.
    '''
    # Initialize lists to store features, labels, and video file paths
    features = []
    labels = []

    # Check if dataset directory exists
    if not os.path.exists(dataset_dir):
        raise FileNotFoundError(f"Dataset directory not found: {dataset_dir}")

    # Iterate through all classes in the classes list
    for class_index, class_name in enumerate(classes_list):
        class_path = os.path.join(dataset_dir, class_name)
        
        # Check if class directory exists
        if not os.path.exists(class_path):
            print(f"Warning: Class directory not found: {class_path}")
            continue

        print(f'Extracting Data of Class: {class_name}')

        # Get the list of video files in the class directory
        files_list = os.listdir(class_path)

        # Limit the number of videos if specified
        if max_videos_per_class is not None:
            files_list = files_list[:max_videos_per_class]

        # Iterate through all video files
        for file_name in files_list:
            video_file_path = os.path.join(class_path, file_name)

            # Extract frames using the updated frames_extraction function
            frames = frames_extraction(video_file_path, sequence_length, image_height, image_width)

            # Skip videos where frame extraction failed
            if frames is None:
                print(f"Skipping video {video_file_path} due to frame extraction failure")
                continue

            # Append the data to respective lists
            features.append(frames)
            labels.append(class_index)

    # Convert lists to numpy arrays
    if not features:
        raise ValueError("No valid videos were processed. Check dataset or parameters.")
    
    features = np.asarray(features)
    labels = np.array(labels)

    print(f"Dataset created with {len(features)} videos")
    print(f"Features shape: {features.shape}")
    print(f"Labels shape: {labels.shape}")

    return features, labels

In [9]:
# RUN Create the dataset with explicit parameters
try:
    features, labels = create_dataset(
        dataset_dir=DATASET_DIR,
        classes_list=CLASSES_LIST,
        sequence_length=SEQUENCE_LENGTH,
        image_height=IMAGE_HEIGHT,
        image_width=IMAGE_WIDTH,
        
        # Limit to 10 videos per class to manage memory
        max_videos_per_class=MAX_VIDEO_PER_CLASS  
    )
except FileNotFoundError as e:
    print(f"Error: {e}")
except ValueError as e:
    print(f"Error: {e}")
except Exception as e:
    print(f"Unexpected error: {e}")

Extracting Data of Class: HorseRace
Extracting Data of Class: VolleyballSpiking
Extracting Data of Class: Biking
Extracting Data of Class: TaiChi
Extracting Data of Class: Punch
Extracting Data of Class: BreastStroke
Dataset created with 600 videos
Features shape: (600, 15, 224, 224, 3)
Labels shape: (600,)


In [10]:

# Define augmentation functions
def horizontal_flip(frames):
    # Flip each frame in the sequence
    print("flipping...")
    return [cv2.flip(frame, 1) for frame in frames]

def random_shear_frames(frames):
    print("shearing...")
    # Shear each frame in the sequence
    return [random_shear(frame) for frame in frames]

def random_shear(frame):
    shear_x = random.uniform(-0.09, 0.09)
    shear_y = random.uniform(0.1, 0.3)
    height, width = frame.shape[:2]
    M = np.float32([[1, shear_x, 0], [shear_y, 1, 0]])
    return cv2.warpAffine(frame, M, (width, height))

def salt_and_pepper_noise_frames(frames):
    print("adding noise...")
    # Add salt and pepper noise to each frame in the sequence
    return [salt_and_pepper_noise(frame) for frame in frames]

def salt_and_pepper_noise(frame, amount=0.02):
    output = np.copy(frame)
    total_pixels = frame.size
    num_salt = int(total_pixels * amount)
    num_pepper = int(total_pixels * amount)
    
    for _ in range(num_salt):
        i = random.randint(0, frame.shape[0] - 1)
        j = random.randint(0, frame.shape[1] - 1)
        output[i, j] = 255  # Salt (white pixel)
    
    for _ in range(num_pepper):
        i = random.randint(0, frame.shape[0] - 1)
        j = random.randint(0, frame.shape[1] - 1)
        output[i, j] = 0  # Pepper (black pixel)
    
    return output


In [11]:
len(features)

600

In [12]:
from tensorflow.keras.utils import Sequence

# Define your data generator
class VideoDataGenerator(Sequence):
    def __init__(self, video_frames, labels, batch_size, augmentations=None):
        self.video_frames = video_frames  # List of sequences, where each sequence is a list of frames
        self.labels = labels
        self.batch_size = batch_size
        self.augmentations = augmentations or []  # List of augmentation functions
    
    def __len__(self):
        # Return the number of batches per epoch
        return int(np.ceil(len(self.video_frames) / self.batch_size))
    
    def __getitem__(self, idx):
        # Get batch of video sequences and corresponding labels
        batch_video_frames = self.video_frames[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_labels = self.labels[idx * self.batch_size:(idx + 1) * self.batch_size]
        
        # Apply augmentations to each sequence
        augmented_frames = [self.apply_augmentations(sequence) for sequence in batch_video_frames]
        
        return np.array(augmented_frames), np.array(batch_labels)
    
    def apply_augmentations(self, sequence):
        # Apply augmentations to each frame in the sequence
        augmented_sequence = sequence
        for aug in self.augmentations:
            augmented_sequence = aug(augmented_sequence)  # Apply augmentation to the entire sequence
        return augmented_sequence
    
    def on_epoch_end(self):
        # Shuffle at the end of each epoch if needed
        pass


In [13]:
# combined = zip(labels, features)

# augmented_features = list(features) # 60 -> 61
# augmented_labels = list(labels)

# # video frame = [frame 1, 2 3 4 ]
# print("Started augmentation frames...")

# for label, video_frames in combined:
    
#     # flip
#     augmented_features.append(horizontal_flip(video_frames))
#     augmented_labels.append(label)
    
#     # shear
#     augmented_features.append(random_shear_frames(video_frames))
#     augmented_labels.append(label)
    
#     # noise
#     augmented_features.append(salt_and_pepper_noise_frames(video_frames))
#     augmented_labels.append(label)
    
#     # augment all together
#     # augmented_features.append(augment_all_together(video_frames))
#     # augmented_labels.append(label)
    

#     #del flipped_video

# print("Data augmented successfully.")
# augmented_features = np.asarray(augmented_features)
# augmented_labels = np.asarray(augmented_labels)

# print(f"Old videos {len(features)}, new videos {len(augmented_features)}")
# del features, labels

In [14]:
# input("Press any key to continue...")

In [15]:
def display_keyframes(keyframes):
    for idx, keyframe in enumerate(keyframes):
        # Ensure the keyframe is in the right data type and scale
        if keyframe.dtype != np.uint8:
            keyframe = np.clip(keyframe, 0, 1)  # If normalized
            keyframe = (keyframe * 255).astype(np.uint8)

        plt.figure(figsize=(5, 5))
        plt.imshow(cv2.cvtColor(keyframe, cv2.COLOR_BGR2RGB))
        plt.title(f'Keyframe {idx + 1}')
        plt.axis('off')
        plt.show()

In [16]:
def display_keyframes_side_by_side(keyframes1, keyframes2, max_pairs=None):
    # Use the minimum number of keyframes to avoid index errors
    num_keyframes = min(len(keyframes1), len(keyframes2))
    if max_pairs is not None:
        num_keyframes = min(num_keyframes, max_pairs)

    for idx in range(num_keyframes):
        kf1 = keyframes1[idx]
        kf2 = keyframes2[idx]

        # Convert both keyframes to uint8 and BGR to RGB
        def prepare(frame):
            if frame.dtype != np.uint8:
                frame = np.clip(frame, 0, 1)
                frame = (frame * 255).astype(np.uint8)
            return cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        kf1 = prepare(kf1)
        kf2 = prepare(kf2)

        # Show side-by-side
        plt.figure(figsize=(10, 5))
        plt.subplot(1, 2, 1)
        plt.imshow(kf1)
        plt.title(f'Original Frame - Keyframe {idx + 1}')
        plt.axis('off')

        plt.subplot(1, 2, 2)
        plt.imshow(kf2)
        plt.title(f'Augmented - Keyframe {idx + 1}')
        plt.axis('off')

        plt.tight_layout()
        plt.show()

In [17]:
# display_keyframes_side_by_side(features[0], augmented_features[60])

In [18]:
# RUN Using Keras's to_categorical method to convert labels into one-hot-encoded vectors
one_hot_encoded_labels = to_categorical(labels)

In [19]:
# RUN Split the Data into Train ( 75% ) and Test Set ( 25% ).
features_train, features_test, labels_train, labels_test = train_test_split(features,
                                                                            one_hot_encoded_labels,
                                                                            test_size = TEST_SIZE,
                                                                            shuffle = True,
                                                                            random_state = seed_constant)

In [20]:
# Assuming features_train and labels_train are defined
train_video_frames, val_video_frames = features_train[:int(0.8 * len(features_train))], features_train[int(0.8 * len(features_train)):]
train_labels, val_labels = labels_train[:int(0.8 * len(labels_train))], labels_train[int(0.8 * len(labels_train)):]

# Create the training and validation generators
train_gen = VideoDataGenerator(
    video_frames=train_video_frames, 
    labels=train_labels, 
    batch_size=BATCH_SIZE, 
    #augmentations=[horizontal_flip, random_shear_frames, salt_and_pepper_noise_frames]
)

val_gen = VideoDataGenerator(
    video_frames=val_video_frames, 
    labels=val_labels, 
    batch_size=BATCH_SIZE
)

In [21]:
del features_train, labels_train

In [22]:
# # free space in ram memory
# del augmented_features, augmented_labels

In [23]:
import tensorflow as tf
from tensorflow.keras.applications import Xception
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import TimeDistributed, Flatten, LSTM, Dropout, Dense

def create_xception_lstm_model(sequence_length, image_height, image_width, classes_list=None):
    if classes_list is None:
        raise ValueError("classes_list must be provided to define the output layer size")

    try:
        # Load Xception model with pre-trained ImageNet weights
        print("Loading Xception base model...")
        xception = Xception(
            weights='imagenet',
            include_top=False,
            input_shape=(image_height, image_width, 3),
            name="Xception"
        )
        # Freeze Xception layers
        for layer in xception.layers:
            layer.trainable = False
        
        # Define the Sequential model
        model = Sequential([
            TimeDistributed(
                xception,
                input_shape=(sequence_length, image_height, image_width, 3),
                name="TimeDistributed_Xception"
            ),
            TimeDistributed(Flatten(), name="Flatten"),
            LSTM(64, activation="relu", return_sequences=False, name="LSTM"),
            Dropout(0.2, name="Dropout"),
            Dense(len(classes_list), activation="softmax", name="Output")
        ])
        
        # Print model summary
        print("Model architecture created successfully!")
        model.summary()

        return model

    except Exception as e:
        print(f"Error creating model: {e}")
        return None

In [24]:
# donwload model weights
from tensorflow.keras.applications import Xception
print("Pre-loading Xception weights...")
base_model = Xception(weights='imagenet', include_top=False, input_shape=(IMAGE_HEIGHT, IMAGE_WIDTH, 3))
print("Weights loaded successfully!")

Pre-loading Xception weights...


I0000 00:00:1751192491.137865      35 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15513 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0


83683744/83683744 ━━━━━━━━━━━━━━━━━━━━ 3s 0us/step
Weights loaded successfully!


In [25]:
# RUN
import gc
tf.keras.backend.clear_session()
gc.collect()

0

In [26]:
# Clear previous session to free memory
tf.keras.backend.clear_session()

# Create the model
xlstm_model = create_xception_lstm_model(
    sequence_length=SEQUENCE_LENGTH,
    image_height=IMAGE_HEIGHT,
    image_width=IMAGE_WIDTH,
    classes_list=CLASSES_LIST
)

# Check if model was created successfully
if xlstm_model is None:
    print("Failed to create model. Check error messages above.")
else:
    print("Model Created Successfully!")

Loading Xception base model...
Model architecture created successfully!


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ TimeDistributed_Xception             │ (None, 15, 7, 7, 2048)      │      20,861,480 │
│ (TimeDistributed)                    │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ Flatten (TimeDistributed)            │ (None, 15, 100352)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ LSTM (LSTM)                          │ (None, 64)                  │      25,706,752 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ Dropout (Dropout)                    │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ Output (Dense)                       │ (None, 6)                   │             390 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 46,568,622 (177.65 MB)

 Trainable params: 25,707,142 (98.06 MB)

 Non-trainable params: 20,861,480 (79.58 MB)

Model Created Successfully!


In [27]:
# Plot the structure of the contructed model.
plot_model(xlstm_model, to_file = f'{MODEL_NAME}_model_Plot.png', show_shapes = True, show_layer_names = True)

print(f"{MODEL_NAME} Model Plot saved successfully...")

Xception Model Plot saved successfully...


In [ ]:
# Create an Instance of Early Stopping Callback
early_stopping_callback = EarlyStopping(monitor = 'val_loss', 
                                        patience = 7, 
                                        mode = 'min', 
                                        restore_best_weights = True)

# Compile the model and specify loss function, optimizer and metrics values to the model
xlstm_model.compile(loss = 'categorical_crossentropy', 
                    optimizer = 'Adam', 
                    metrics = ["accuracy"])

# Start training the model.
# = features_train,
#y = labels_train,
#batch_size = BATCH_SIZE,

convlstm_model_training_history = xlstm_model.fit(train_gen,
                                                  epochs = EPOCHS,
                                                  shuffle = True,
                                                  validation_data=val_gen,
                                                  callbacks = [early_stopping_callback])

Epoch 1/50


I0000 00:00:1751192569.917952    2496 service.cc:148] XLA service 0x2688c7a0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1751192569.919002    2496 service.cc:156]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0
I0000 00:00:1751192582.319093    2496 cuda_dnn.cc:529] Loaded cuDNN version 90300
I0000 00:00:1751192593.381687    2496 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


64/64 ━━━━━━━━━━━━━━━━━━━━ 155s 945ms/step - accuracy: 0.2558 - loss: 201.5883 - val_accuracy: 0.3021 - val_loss: 230.7951
Epoch 2/50
64/64 ━━━━━━━━━━━━━━━━━━━━ 30s 472ms/step - accuracy: 0.2689 - loss: 241.6668 - val_accuracy: 0.2292 - val_loss: 488.5793
Epoch 3/50
64/64 ━━━━━━━━━━━━━━━━━━━━ 30s 471ms/step - accuracy: 0.3002 - loss: 443.9450 - val_accuracy: 0.3438 - val_loss: 721.2109
Epoch 4/50
64/64 ━━━━━━━━━━━━━━━━━━━━ 30s 471ms/step - accuracy: 0.3139 - loss: 566.8730 - val_accuracy: 0.4688 - val_loss: 355.7494
Epoch 5/50
64/64 ━━━━━━━━━━━━━━━━━━━━ 30s 476ms/step - accuracy: 0.3332 - loss: 237.3086 - val_accuracy: 0.3646 - val_loss: 129.7928
Epoch 6/50
64/64 ━━━━━━━━━━━━━━━━━━━━ 30s 474ms/step - accuracy: 0.3466 - loss: 156.7970 - val_accuracy: 0.4062 - val_loss: 60.2733
Epoch 7/50
64/64 ━━━━━━━━━━━━━━━━━━━━ 30s 474ms/step - accuracy: 0.3961 - loss: 44.5260 - val_accuracy: 0.5417 - val_loss: 19.4283
Epoch 8/50
64/64 ━━━━━━━━━━━━━━━━━━━━ 30s 474ms/step - accuracy: 0.4915 - loss: 28

In [ ]:
# previous code
model_evaluation_history = xlstm_model.evaluate(features_test, labels_test)

In [ ]:
plt.figure(figsize=(12,6))
plt.subplot(1,2,1)
plt.title("Accuracy",weight="bold",fontsize=20)
plt.plot(convlstm_model_training_history.history["accuracy"])
plt.plot(convlstm_model_training_history.history["val_accuracy"])
plt.legend(["Train","Validation"])
plt.xlabel("Epochs",weight="bold")
plt.ylabel("Accuracy",weight="bold")
plt.subplot(1,2,2)
plt.title("loss",weight="bold",fontsize=20)
plt.plot(convlstm_model_training_history.history["loss"])
plt.plot(convlstm_model_training_history.history["val_loss"])
plt.legend(["Train","Validation"])
plt.xlabel("Epochs",weight="bold")
plt.ylabel("loss",weight="bold")
plt.tight_layout()
plt.show()

In [ ]:
# Evaluate the trained model.

# Create a dataset from your test features and labels
test_dataset = tf.data.Dataset.from_tensor_slices((features_test, labels_test))
test_dataset = test_dataset.batch(8)  # Try 4, 8, 16 depending on your memory

# Evaluate using the dataset
model_evaluation_history = xlstm_model.evaluate(test_dataset)

In [ ]:
# # Get the loss and accuracy from model_evaluation_history.
# model_evaluation_loss, model_evaluation_accuracy = model_evaluation_history

# # Define the string date format.
# # Get the current Date and Time in a DateTime Object.
# # Convert the DateTime object to string according to the style mentioned in date_time_format string.
# date_time_format = '%Y_%m_%d__%H_%M_%S'
# current_date_time_dt = dt.datetime.now()
# current_date_time_string = dt.datetime.strftime(current_date_time_dt, date_time_format)

# # Define a useful name for our model to make it easy for us while navigating through multiple saved models.
# model_file_name = f'xlstm_model___Date_Time_{current_date_time_string}___Loss_{model_evaluation_loss}___Accuracy_{model_evaluation_accuracy}.h5'

# # Save your Model.
# xlstm_model.save(model_file_name)

In [ ]:
def plot_metric(model_training_history, metric_name_1, metric_name_2, plot_name):
    '''
    This function will plot the metrics passed to it in a graph.
    Args:
        model_training_history: A history object containing a record of training and validation
                                loss values and metrics values at successive epochs
        metric_name_1:          The name of the first metric that needs to be plotted in the graph.
        metric_name_2:          The name of the second metric that needs to be plotted in the graph.
        plot_name:              The title of the graph.
    '''

    # Get metric values using metric names as identifiers.
    metric_value_1 = model_training_history.history[metric_name_1]
    metric_value_2 = model_training_history.history[metric_name_2]

    # Construct a range object which will be used as x-axis (horizontal plane) of the graph.
    epochs = range(len(metric_value_1))

    # Plot the Graph.
    plt.plot(epochs, metric_value_1, 'blue', label = metric_name_1)
    plt.plot(epochs, metric_value_2, 'red', label = metric_name_2)

    # Add title to the plot.
    plt.title(str(plot_name))

    # Add legend to the plot.
    plt.legend()

In [ ]:
# Visualize the training and validation loss metrices.
plot_metric(convlstm_model_training_history, 'loss', 'val_loss', 'Total Loss vs Total Validation Loss')

In [ ]:
# Visualize the training and validation accuracy metrices.
plot_metric(convlstm_model_training_history, 'accuracy', 'val_accuracy', 'Total Accuracy vs Total Validation Accuracy')